For å kjøre de følgende notatbøkene, hvis du ikke har gjort det ennå, må du sette openai-nøkkelen inne i .env-filen som `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Deretter skal vi laste inn Embedding-indeksen i en Pandas Dataframe. Embedding-indeksen er lagret i en JSON-fil kalt `embedding_index_3m.json`. Embedding-indeksen inneholder embeddingene for hver av YouTube-transkripsjonene frem til slutten av oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Deretter skal vi lage en funksjon kalt `get_videos` som søker i Embedding-indeksen etter spørringen. Funksjonen vil returnere de 5 beste videoene som er mest like spørringen. Funksjonen fungerer som følger:

1. Først opprettes en kopi av Embedding-indeksen.
2. Deretter beregnes Embedding for spørringen ved hjelp av OpenAI Embedding API.
3. Så opprettes en ny kolonne i Embedding-indeksen kalt `similarity`. `similarity`-kolonnen inneholder cosinuslikheten mellom spørringens Embedding og Embedding for hvert videosegment.
4. Deretter filtreres Embedding-indeksen etter `similarity`-kolonnen. Embedding-indeksen filtreres for kun å inkludere videoer som har en cosinuslikhet større enn eller lik 0,75.
5. Til slutt sorteres Embedding-indeksen etter `similarity`-kolonnen, og de 5 beste videoene returneres.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Denne funksjonen er veldig enkel, den skriver bare ut resultatene av søkespørringen.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Først lastes Embedding-indeksen inn i en Pandas Dataframe.  
2. Deretter blir brukeren bedt om å skrive inn en forespørsel.  
3. Så kalles funksjonen `get_videos` for å søke i Embedding-indeksen etter forespørselen.  
4. Til slutt kalles funksjonen `display_results` for å vise resultatene til brukeren.  
5. Brukeren blir deretter bedt om å skrive inn en ny forespørsel. Denne prosessen fortsetter til brukeren skriver `exit`.  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.no.png)  

Du vil bli bedt om å skrive inn en forespørsel. Skriv inn en forespørsel og trykk enter. Applikasjonen vil returnere en liste over videoer som er relevante for forespørselen. Applikasjonen vil også returnere en lenke til stedet i videoen hvor svaret på spørsmålet finnes.  

Her er noen forespørsler du kan prøve:  

- Hva er Azure Machine Learning?  
- Hvordan fungerer konvolusjonelle nevrale nettverk?  
- Hva er et nevralt nettverk?  
- Kan jeg bruke Jupyter Notebooks med Azure Machine Learning?  
- Hva er ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på originalspråket skal anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
